In [1]:
 !pip install accelerate>=0.20.1
 !pip install transformers
 !pip install pytorch-lightning
 !pip install sentencepiece
 !pip install sacremoses

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 56.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.18.0
    Uninstalling huggingface-hub-0.18.0:
      Successfully uninstalled huggingface-hub-0.18.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.6/774.6 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 55.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895241 sha256=596747df9fd87fdb11d13d2303

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!git clone https://github.com/seokyoung96/DA.git

Cloning into 'DA'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 37 (delta 13), reused 30 (delta 11), pack-reused 0
Receiving objects: 100% (37/37), 38.99 KiB | 13.00 MiB/s, done.
Resolving deltas: 100% (13/13), done.


In [4]:
import pandas as pd
df_train = pd.read_csv('/content/DA/transformer/df_100_train.csv')
df_val = pd.read_csv('/content/DA/transformer/df_100_valid.csv')

In [5]:
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
    MarianTokenizer,
    MarianMTModel,
    MarianConfig,
)

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import random

model_name = "Helsinki-NLP/opus-mt-ko-en"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [6]:
import sys
sys.path.append("/content/DA/transformer")

In [7]:
import translate_model

In [8]:
custom_dataset = translate_model.CustomDataset(tokenizer=tokenizer)

dataset_train = custom_dataset.get_inputs(df_train)
dataset_val = custom_dataset.get_inputs(df_val)

In [9]:
df_train.iloc[:, 1]

0     The National Institute of Environmental Resear...
1     "There are some projects that are not listed o...
2     Writer Yoo was released after receiving an ind...
3     The research institute said, “As a result of d...
4     IT foreign media such as Burgie and Engadget a...
                            ...                        
95    The Criminal Chamber 3 of Seoul High Court (Pr...
96    I didn't expect such a good fortune came to me...
97    An average of 6.11 million people are expected...
98    When Hyun Joo-yeop made a phone call, a staff ...
99    In the preview video of the 7th episode, which...
Name: 번역문, Length: 100, dtype: object

In [10]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = translate_model.CustomModel(custom_dataset, model_name, device, tokenizer)

model_path = "/content/drive/MyDrive/model/"

num_train_epochs = 10
batch_size = 10
step = 500

training_args = Seq2SeqTrainingArguments(
    output_dir=model_path, #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=num_train_epochs, # number of training epochs
    per_device_train_batch_size=batch_size, # batch size for training
    per_device_eval_batch_size=batch_size,  # batch size for evaluation
    eval_steps=step, # Number of update steps between two evaluations.
    save_steps=step, # after # steps model is saved
    logging_steps=step,
    prediction_loss_only=True,
    evaluation_strategy="steps",
    save_total_limit=3
    )

performance_max = 0

for i in range(10):
    model.train(training_args, df_train, df_val)
    performance = model.test_accuracy(df_train, batch_size=10)
    print(f'{i+1} epoch performance : {performance}')

    if performance_max < performance :
        performance_max = performance
        print(f'{i+1} epoch is best model\n')

Step,Training Loss,Validation Loss


### 1 batch start ###
1 batch performance : 0.0

### 2 batch start ###
2 batch performance : 0.0

### 3 batch start ###
3 batch performance : 0.0

### 4 batch start ###
4 batch performance : 0.0

### 5 batch start ###
5 batch performance : 0.0

### 6 batch start ###
6 batch performance : 0.0

### 7 batch start ###
7 batch performance : 0.0

### 8 batch start ###
8 batch performance : 0.0

### 9 batch start ###
9 batch performance : 0.0

### 10 batch start ###
10 batch performance : 0.0

final performance : 0.0
1 epoch performance : 0.0


Step,Training Loss,Validation Loss


### 1 batch start ###
1 batch performance : 0.0

### 2 batch start ###
2 batch performance : 0.0

### 3 batch start ###
3 batch performance : 0.0

### 4 batch start ###
4 batch performance : 0.0

### 5 batch start ###
5 batch performance : 0.0

### 6 batch start ###
6 batch performance : 0.0

### 7 batch start ###
7 batch performance : 0.0

### 8 batch start ###
8 batch performance : 0.0

### 9 batch start ###
9 batch performance : 0.0

### 10 batch start ###
10 batch performance : 0.0

final performance : 0.0
2 epoch performance : 0.0


Step,Training Loss,Validation Loss


...

### 1 batch start ###
1 batch performance : 0.0

### 2 batch start ###
2 batch performance : 0.0

### 3 batch start ###
3 batch performance : 0.0

### 4 batch start ###
4 batch performance : 0.3

### 5 batch start ###
5 batch performance : 0.0

### 6 batch start ###
6 batch performance : 0.0

### 7 batch start ###
7 batch performance : 0.2

### 8 batch start ###
8 batch performance : 0.1

### 9 batch start ###
9 batch performance : 0.1

### 10 batch start ###
10 batch performance : 0.0

final performance : 0.06999999999999999
8 epoch performance : 0.06999999999999999
8 epoch is best model



Step,Training Loss,Validation Loss


### 1 batch start ###
1 batch performance : 0.2

### 2 batch start ###
2 batch performance : 0.2

### 3 batch start ###
3 batch performance : 0.3

### 4 batch start ###
4 batch performance : 0.3

### 5 batch start ###
5 batch performance : 0.4

### 6 batch start ###
6 batch performance : 0.2

### 7 batch start ###
7 batch performance : 0.1

### 8 batch start ###
8 batch performance : 0.3

### 9 batch start ###
9 batch performance : 0.2

### 10 batch start ###
10 batch performance : 0.2

final performance : 0.24000000000000005
9 epoch performance : 0.24000000000000005
9 epoch is best model



Step,Training Loss,Validation Loss


### 1 batch start ###
1 batch performance : 0.2

### 2 batch start ###
2 batch performance : 0.5

### 3 batch start ###
3 batch performance : 0.2

### 4 batch start ###
4 batch performance : 0.2

### 5 batch start ###
5 batch performance : 0.3

### 6 batch start ###
6 batch performance : 0.3

### 7 batch start ###
7 batch performance : 0.2

### 8 batch start ###
8 batch performance : 0.2

### 9 batch start ###
9 batch performance : 0.1

### 10 batch start ###
10 batch performance : 0.3

final performance : 0.25
10 epoch performance : 0.25
10 epoch is best model

